### Bibertomograph [Bwinf44 2025](https://bwinf.de/bundeswettbewerb/44/) - A4

[Video]()


<img src="aufgabe1.png" alt="Bild 1" style="display:inline-block; width:250px;">
<img src="aufgabe2.png" alt="Bild 2" style="display:inline-block; width:250px;">


#### Eingabedaten

Jede Datei beschreibt ein Fläche und ist so aufgebaut:

- 1. Zeile: Anzahl n der Zeilen und Spalten.
- 2. Zeile: Anzahl der gefüllten Felder pro Spalte von links nach rechts
- 3. Zeile: Anzahl der gefüllten Felder pro Zeile von oben nach unten
- 4. Zeile: Anzahl der gefüllten Felder pro blauer diagonaler Sichtlinie von oben links bis unten rechts
- 5. Zeile: Anzahl der gefüllten Felder pro gelber diagonaler Sichtlinien von unten links nach oben rechts
Die Datei tomograph00.txt entspricht dem Beispiel aus der Aufgabenstellung. Achtung! Auf dem gedruckten Aufgabenblatt ist das Bild fehlerhaft. Das richtige Bild ist auf dem digitalen Aufgabenblatt zu finden.


Hier eine Beispieleingabe:

    2
    1 0
    0 1
    0 1 0
    1 0 0  

<img src='beispiel1.png' width='400'>

In dem Beispiel wird auf einem Feld bestehend aus zwei Zeilen und Spalten gespielt.
In der ersten Spalte gibt es ein gefülltes Feld, in der zweite Spalte keins. In der ersten Zeile gibt es kein gefülltes Feld, in der zweiten Zeile eins.
Die blauen diagonalen Sichtlinien werden von oben links nach unten rechts gelesen. Auf der ersten Sichtlinie liegt kein gefülltes Feld, auf der zweiten Linien eins und auf der dritten wieder keins.
Die gelben diagonalen Sichtlinien werden von unten links nach oben rechts gelesen. Auf der ersten Sichtlinie liegt ein gefülltes Feld, auf der zweiten und dritten keins.


Aus den Angaben der Beispieleingabe ergibt sich als eindeutige Lösung die Figur auf der rechten Seite.

Die Datei tomograph00.txt entspricht dem Beispiel aus der Aufgabenstellung. Achtung! Auf dem gedruckten Aufgabenblatt ist das Bild fehlerhaft. Das richtige Bild ist auf dem digitalen Aufgabenblatt zu finden.

#### Beispieldaten

In [ ]:
# Anschauen der Beispieldaten 00-10
nr = '10'
print(f'Beispiel {nr}:')
f = open(f'beispieldaten/tomograph{nr}.txt')
print(f.read().strip())
f.close()

Beispiel 02:
4
3 2 2 2
1 2 4 2
0 1 3 2 1 1 1
1 1 2 3 1 1 0


#### Lösungsidee

Wir müssen für jedes Feld entscheiden, ob es gefüllt ist oder nicht. Dabei müssen wir die Bedingungen beachten, die durch Azzes Beobachtung gegeben sind. Wir nutzen dazu einen Solver, dem wir die Bedingungen in geeigneter Weise mitteilen. Als Solver nutzen wir Minizinc.

<img src='bild1.png' width='600'>



In [ ]:
var 0..1: a1; 
var 0..1: a2; 
var 0..1: a3; 
var 0..1: a4; 

constraint a1 + a3 = 1;
constraint a2 + a4 = 0;
constraint a1 + a2 = 0;
constraint a3 + a4 = 1;
constraint a1 = 0;
constraint a2 + a3 = 1;
constraint a4 = 0;
constraint a3 = 1;
constraint a4 + a1 = 0;
constraint a2 = 0;

solve satisfy;

#### Variablennamen

    sp = Liste mit der Anzahl 1er für die Spalten
    ze = Liste mit der Anzahl 1er für die Zeilen
    d1 = Liste mit der Anzahl 1er für die 1.Diagonalengruppe
    d2 = Liste mit der Anzahl 1er für die 2.Diagonalengruppe

    spalten = Liste mit den Indizes der Spalten
    zeilen = Liste mit den Indizes der Zeilen
    diagonals1 = Liste mit den Indizes der 1.Diagonalengruppe
    diagonals2 = Liste mit den Indizes der 2.Diagonalengruppe




Durch die Nutzung von Listen (Arrays) machen wir das Programm von den Beispieldaten unabhängiger. Wir schreiben mit einem Python-Programm die Eingabedaten in die Datei *tomograph.dzn*. Das eigentliche Programm *tomograph.mzn* bleibt für alle Fälle gleich.

In [5]:
%%writefile tomograph.dzn
n=2;
sp = [1, 0];
ze = [0, 1];
d1 = [0, 1, 0];
d2 = [1, 0, 0];
spalten = [[0, 2], [1, 3]];
zeilen = [[0, 1], [2, 3]];
diagonals1 = [[0], [1, 2], [3]];
diagonals2 = [[2], [3, 0], [1]];


Overwriting tomograph.dzn


In [10]:
%%writefile tomograph.mzn
int: n;
array[1..n] of int: sp;
array[1..n] of int: ze;
array[1..2*n-1] of int: d1;
array[1..2*n-1] of int: d2;
array[1..n] of array[int] of int: spalten;
array[1..n] of array[int] of int: zeilen;
array[1..2*n-1] of array[int] of int: diagonals1;
array[1..2*n-1] of array[int] of int: diagonals2;

% ----- Decision variables -----
array[0..n*n-1] of var bool: x;

% ----- Constraints -----
% Spalten
constraint
    forall(i in 1..n)(
        sum(j in 1..n)( x[spalten[i][j]] ) = sp[i]
    );

% Zeilen
constraint
    forall(i in 1..n)(
        sum(j in 1..n)( x[zeilen[i][j]] ) = ze[i]
    );

% Diagonalen1 
constraint
    forall(i in 1..2*n-1)(
        sum(j in index_set(diagonals1[i]))( x[diagonals1[i][j]] ) = d1[i]
    );

% Diagonalen2
constraint
    forall(i in 1..2*n-1)(
        sum(j in index_set(diagonals2[i]))( x[diagonals2[i][j]] ) = d2[i]
    );

% ----- Solve -----
solve satisfy;

% ----- Output -----
output [ 
    if fix(x[r* n + c]) == 1 then "1 " else "0 " endif ++
    if c = n-1 then "\n" else "" endif
    | r in 0..n-1, c in 0..n-1
];

Overwriting tomograph.mzn


#### Eingabe für Minizinc

In [69]:
nr = '04' 
f = open('beispieldaten/tomograph'+nr+'.txt')
n = int(f.readline())

sp = [int(x) for x in f.readline().split()]
ze = [int(x) for x in f.readline().split()]
d1 = [int(x) for x in f.readline().split()]
d2 = [int(x) for x in f.readline().split()]
f.close()

def get_spalten(n):
    tmp = []
    for i in range(n):
        tmp.append([i+n*j for j in range(n)])
    return tmp

def get_zeilen(n):
    tmp = []
    for i in range(n):
        tmp.append([i*n+j for j in range(n)])
    return tmp

def get_diagonals1(n):
    diagonals = []
    # Startpunkte in der ersten Zeile
    for start_col in range(n):
        diag = []
        row, col = 0, start_col
        while row < n and col >= 0:
            diag.append((n*row + col))
            row += 1
            col -= 1
        diagonals.append(diag)

    # Startpunkte in der rechten Spalte 
    for start_row in range(1, n):
        diag = []
        row, col = start_row, n-1
        while row < n and col >= 0:
            diag.append((n*row + col))
            row += 1
            col -= 1
        diagonals.append(diag)

    return diagonals

def get_diagonals2(n):
    diagonals = []
    # Startpunkte in der unteren Zeile
    for start_col in range(n):
        diag = []
        row, col = n-1, start_col
        while row >= 0 and col >= 0:
            diag.append((n*row + col))
            row -= 1
            col -= 1
        diagonals.append(diag)

    # Startpunkte in der rechten Spalte (außer unten rechts, weil schon enthalten)
    for start_row in range(n-2, -1, -1):
        diag = []
        row, col = start_row, n-1
        while row >= 0 and col >= 0:
            diag.append((n*row + col))
            row -= 1
            col -= 1
        diagonals.append(diag)
    return diagonals
    

diagonals1 = get_diagonals1(n)
diagonals2 = get_diagonals2(n)
spalten = get_spalten(n)
zeilen = get_zeilen(n)

f = open('tomograph.dzn',encoding='utf-8',mode='w')       

print(f'{n=};', file=f)
print(f'{sp = };', file=f)
print(f'{ze = };', file=f)
print(f'{d1 = };' ,file=f)
print(f'{d2 = };', file=f)

print(f'{spalten = };', file=f)
print(f'{zeilen = };', file=f)
print(f'{diagonals1 = };', file=f)
print(f'{diagonals2 = };', file=f)
f.close()

In [70]:
import subprocess

MINIZINC = r"C:\Program Files\MiniZinc\minizinc.exe"
MODEL = "tomograph.mzn"
DATA = "tomograph.dzn"

cmd = [
    MINIZINC,
    "--solver", "gecode",
    MODEL,
    DATA
]

result = subprocess.run(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print(f'Beispiel {nr}:')
print(result.stdout)

Beispiel 04:
0 0 1 0 0 
1 1 0 0 1 
0 1 1 1 0 
1 1 0 0 1 
0 0 1 0 0 
----------



Um herauszufingen, welche Positionen nicht variabel belegt werden können, testen wir für jede Position aus, welche Belegung eine Lösung hat. Dazu fügen wir in unser Model ein weiter constraint ein und gehen mit einer Schleife über die verschiedenen Möglichkeiten.

In [ ]:
int: p;   % getestete Position
int: v;   % 0 oder 1
constraint x[p] = v;

In [71]:
import subprocess

MINIZINC = r"C:\Program Files\MiniZinc\minizinc.exe"
MODEL = "tomograph_pos.mzn"
DATA = "tomograph.dzn"

def is_sat(p, v):
    cmd = [
        MINIZINC,
        "--solver", "gecode",
        MODEL,
        DATA,
        f"-Dp={p}",
        f"-Dv={v}"
    ]

    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    return "UNSATISFIABLE" not in result.stdout

always_0 = []
always_1 = []

for p in range(n * n):
    sat0 = is_sat(p, 0)
    sat1 = is_sat(p, 1)

    if sat0 and not sat1:
        always_0.append(p)
    elif sat1 and not sat0:
        always_1.append(p)

print(f'Beispiel {nr}:')
print(result.stdout.strip())

a = ['?'] * (n*n)
for i in always_0:
    a[i]=0
for i in always_1:
    a[i]= 1

for i in range(n*n):
    if i % n == 0:
        print()
    print(a[i],end=' ')
    


Beispiel 04:
0 0 1 0 0 
1 1 0 0 1 
0 1 1 1 0 
1 1 0 0 1 
0 0 1 0 0 
----------

0 ? ? ? 0 
? ? ? ? ? 
? ? 1 ? ? 
? ? ? ? ? 
0 ? ? ? 0 

#### Das ganze Programm

In [65]:
import subprocess
nr = '00' 
f = open('beispieldaten/tomograph'+nr+'.txt')
n = int(f.readline())

sp = [int(x) for x in f.readline().split()]
ze = [int(x) for x in f.readline().split()]
d1 = [int(x) for x in f.readline().split()]
d2 = [int(x) for x in f.readline().split()]
f.close()

def get_spalten(n):
    tmp = []
    for i in range(n):
        tmp.append([i+n*j for j in range(n)])
    return tmp

def get_zeilen(n):
    tmp = []
    for i in range(n):
        tmp.append([i*n+j for j in range(n)])
    return tmp

def get_diagonals1(n):
    diagonals = []
    # Startpunkte in der ersten Zeile
    for start_col in range(n):
        diag = []
        row, col = 0, start_col
        while row < n and col >= 0:
            diag.append((n*row + col))
            row += 1
            col -= 1
        diagonals.append(diag)

    # Startpunkte in der rechten Spalte 
    for start_row in range(1, n):
        diag = []
        row, col = start_row, n-1
        while row < n and col >= 0:
            diag.append((n*row + col))
            row += 1
            col -= 1
        diagonals.append(diag)

    return diagonals

def get_diagonals2(n):
    diagonals = []
    # Startpunkte in der unteren Zeile
    for start_col in range(n):
        diag = []
        row, col = n-1, start_col
        while row >= 0 and col >= 0:
            diag.append((n*row + col))
            row -= 1
            col -= 1
        diagonals.append(diag)

    # Startpunkte in der rechten Spalte (außer unten rechts, weil schon enthalten)
    for start_row in range(n-2, -1, -1):
        diag = []
        row, col = start_row, n-1
        while row >= 0 and col >= 0:
            diag.append((n*row + col))
            row -= 1
            col -= 1
        diagonals.append(diag)
    return diagonals
    

diagonals1 = get_diagonals1(n)
diagonals2 = get_diagonals2(n)
spalten = get_spalten(n)
zeilen = get_zeilen(n)

f = open('tomograph.dzn',encoding='utf-8',mode='w')       

print(f'{n=};', file=f)
print(f'{sp = };', file=f)
print(f'{ze = };', file=f)
print(f'{d1 = };' ,file=f)
print(f'{d2 = };', file=f)

print(f'{spalten = };', file=f)
print(f'{zeilen = };', file=f)
print(f'{diagonals1 = };', file=f)
print(f'{diagonals2 = };', file=f)
f.close()

MINIZINC = r"C:\Program Files\MiniZinc\minizinc.exe"
MODEL = "tomograph.mzn"
DATA = "tomograph.dzn"

cmd = [
    MINIZINC,
    "--solver", "gecode",
    MODEL,
    DATA
]

result = subprocess.run(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

MODEL = "tomograph_pos.mzn"

def is_sat(p, v):
    cmd = [
        MINIZINC,
        "--solver", "gecode",
        MODEL,
        DATA,
        f"-Dp={p}",
        f"-Dv={v}"
    ]

    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    return "UNSATISFIABLE" not in result.stdout

always_0 = []
always_1 = []

for p in range(n * n):
    sat0 = is_sat(p, 0)
    sat1 = is_sat(p, 1)

    if sat0 and not sat1:
        always_0.append(p)
    elif sat1 and not sat0:
        always_1.append(p)

print(f'Beispiel {nr}:')
print(result.stdout.strip())

a = ['?'] * (n*n)
for i in always_0:
    a[i]=0
for i in always_1:
    a[i]= 1

if '?' in a:  
    for i in range(n*n):
        if i % n == 0:
            print()
        print(a[i],end=' ')
else:
    print('Lösung eindeutig')

Beispiel 00:
0 0 1 1 1 0 0 0 
0 0 1 0 1 0 0 0 
0 0 1 1 1 0 0 0 
1 0 0 1 0 0 0 0 
0 1 1 1 1 1 1 0 
0 0 0 1 0 0 1 0 
0 0 0 1 0 0 0 1 
0 0 1 0 1 0 0 0 
----------
Lösung eindeutig
